In [ ]:
- First try with random during generator.
- Handle the case where max(mcc) is not unique. Use pickle label file.

In [ ]:
#Calculate locations of 1297 cells
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef as mcc
import pickle

bdtnp = pd.read_csv('../data/binarized_bdtnp.csv', squeeze=True)
dge = pd.read_csv('../data/dge_binarized_distMap.csv', squeeze=True)
#print(dge.shape) #(84, 1297)
#bdtnp.shape #(3039, 84)
#print(dge['GTACTAATTACN_2'].dot(bdtnp.iloc[0])) #15
#print(dge['GTACTAATTACN_2'].dtypes) #dtype('int64')
#print(dge[col].shape) #(84,)
#dge[col] * bdtnp

ind = {}
#Columns in DGE (RNA_seq data) are cell values for the 84 in-citu genes only.
i = 0
data = pd.DataFrame()
for col in dge:
    print(i, ' ', end="")
    i = i + 1
    my_list = list( mcc(dge[col], bdtnp.iloc[i]) for i in range(3039))
    ind[col] = np.argwhere(my_list == np.amax(my_list)) #Index in bdtnp is zero based.
    #ind[col] = max(dge[col] * bdtnp.iloc[i]) #Test
    data = pd.concat([data, pd.DataFrame([[gene, np.ndarray.flatten(loc)]], columns=['genes', 'locations'])])
    print(ind[col])

data.to_csv('data/labels_raw.csv')
output = open('labels.pkl', 'wb')
pickle.dump(ind, output)
output.close()

In [142]:
ind

{'GTACTAATTACN_2': array([[133]], dtype=int64),
 'AGGCTAATGGAC_3': array([[781]], dtype=int64),
 'TAGACAAAAGCT_4': array([[1704],
        [2123]], dtype=int64),
 'TCTACTAGTGTN_5': array([[52],
        [90]], dtype=int64),
 'CCGTGCTGAACA_3': array([[1440]], dtype=int64),
 'GAGAAAATGAAG_7': array([[2597]], dtype=int64),
 'GCCTAGTGACGT_4': array([[258]], dtype=int64),
 'CAGTATCGAGAN_2': array([[113]], dtype=int64),
 'AGATTACCCTTC_6': array([[1254]], dtype=int64),
 'AACCCCACGCTC_2': array([[33]], dtype=int64),
 'GTACTGCCAGAG_4': array([[795]], dtype=int64),
 'CTAATAAATATT_3': array([[1200],
        [1202],
        [1203]], dtype=int64),
 'AAAGTCTAAGCG_3': array([[1152]], dtype=int64),
 'TTCGTTAAATAA_3': array([[2797]], dtype=int64),
 'AAGGAGAGAGTN_2': array([[186],
        [225]], dtype=int64),
 'CACAATTGGCGT_7': array([[2889]], dtype=int64),
 'CGTGAATTAAGG_4': array([[1287],
        [2943]], dtype=int64),
 'GCATCCCCTACA_3': array([[1183]], dtype=int64),
 'GGAGCAACTAAN_5': array([[1111]], 

In [ ]:
pkl_file = open('data/labels.pkl', 'rb')
ind_load = pickle.load(pkl_file)
pkl_file.close()
data_ind = pd.DataFrame(list(ind_load.items()))
data_ind.drop([0], axis=1, inplace=True)
data_ind[1] = [np.ndarray.flatten(data_ind[1][i]) for i in range(len(data_ind))]
#data_ind.head()
d_true = []
for i in range(len(data_ind)):
    for j in range(len(data_ind.iloc[i].iloc[0])):
        d_true.append((i,data_ind.iloc[i].iloc[0][j],1))
d_true

In [ ]:
#Create the true label pairs. Left: d-array, right: b-array 
#Labels start from 1 in the original file. They indicate a specific row in b table.
print(time.ctime(),'Create true list of tuples')
d['label'] = labels['label'] - 1
d['one'] = 1
d_true = list(zip(d.index, d.label, d.one))
d = d.drop(['one', 'label'], 1)

In [22]:
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, Dropout, Lambda, Activation, BatchNormalization
import time

b = pd.read_csv('bdtnp.csv')
d = pd.read_csv('dge_raw.csv', index_col=0, header=None, encoding='ISO-8859-1').T
labels = pd.read_csv('labels.csv', index_col=0, header=None).T
b['label'] = b.index + 1
d = d.div(d.sum(axis=1), axis=0)
d['label'] = labels['label']

#Create the true label dataframe
#dge_true = pd.merge(dge, bdtnp, on='label', how='inner')
#Create the false label dataframe
#dge.ix[:,0:-1]
#dge[dge.columns[1:-1]]
#dge.iloc[:,0:-1]
#bdtnp['label'] = 0 #bdtnp.index + 1
#dge['label'] = 0
#dge_false = pd.merge(dge, bdtnp, on='label')

In [ ]:
import random
from random import shuffle
import numpy as np

#Output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`.
def generate_array(batch_size=100):
    while True:
        X = np.empty((batch_size, 9008))
        y = np.empty((batch_size), dtype=int)
        b_shuf = [i for i in range(len(b))]
        shuffle(b_shuf)
        d_shuf = [j for j in range(len(d))]
        shuffle(d_shuf)
        batch = 0
        for i in b_shuf:
             for j in d_shuf:
                b_row = b.iloc[i]
                d_row = d.iloc[j]
                eq = 1 if d_row.label == b_row.label else 0
                if (eq == 0):
                    rand = random.randint(1,101)
                    #Return only 1% of the cases of negative class
                    if rand > 1:
                        continue
                b_row = b_row.drop('label')
                d_row = d_row.drop('label')
                row = b_row.append(d_row)
                X[batch] = row
                y[batch] = eq
                batch = batch + 1
                if (batch == batch_size):
                    batch = 0
                    yield(X,y)

myg = generate_array()
for i in myg:
     #print(i[0].shape)
     print(i[0][0])

In [16]:
#Model build
print(time.strftime("%H:%M:%S"), ' Model build')

model = Sequential()
model.add(Dense(84, activation='relu', input_dim=9008)) #9008
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
print(model.summary())

20:58:22  Model build
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 84)                756756    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 85        
Total params: 756,841
Trainable params: 756,841
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print(time.strftime("%H:%M:%S"), ' Fit')
tbCallBack = keras.callbacks.TensorBoard(log_dir='.', histogram_freq=0, write_graph=True, write_images=True)
history = model.fit_generator(generate_array(),
                              steps_per_epoch=395455,
                              epochs=5,
                              verbose=1,
                              class_weight={0:1, 1:400},
                              callbacks=[tbCallBack])

# GPU tests

In [ ]:
tf.test.gpu_device_name()
#'/device:GPU:0'

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

ln: cannot remove ‘/usr/bin/nvidia-smi’: Permission denied
Requirement already satisfied: gputil in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (1.3.0)
Requirement already satisfied: numpy in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from gputil) (1.15.1)
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.5 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.3 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: psutil in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (5.4.5)
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.5 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.3 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: humanize in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (0.5.1)
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.5 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.3 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Gen RAM Free: 60.0 GB  | Proc size: 3.2 GB
GPU RAM Free: 734MB | Used: 15418MB | Util  95% | Total 16152MB